In [1]:
%matplotlib inline
from __future__ import print_function, division
from io import open
import string
import re
import random
import sklearn 
from sklearn.metrics import roc_auc_score
import plotly.plotly as py 
import plotly.graph_objs as go
import torch
import torch.nn as nn
import torch.autograd as autograd
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import torch.utils.data as Data

use_cuda = torch.cuda.is_available()
torch.cuda.set_device(0)

import sys, random
import numpy as np
try:
    import cPickle as pickle
except:
    import pickle

from torchviz import make_dot, make_dot_from_trace

# for windows only    
#import os
#os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'


In [2]:
use_cuda

True

In [3]:
'''
# H143 no encounter data input_size= 15816
in_size=15816
train_sl = pickle.load(open('pdata_3hosp/h143_train', 'rb'), encoding='bytes')
valid_sl = pickle.load(open('pdata_3hosp/h143_valid', 'rb'), encoding='bytes')
test_sl = pickle.load(open('pdata_3hosp/h143_test', 'rb'), encoding='bytes')

# H143 with encounter data input_size= 18250
in_size= 18250
train_sl = pickle.load(open('/data/projects/py_ehr_2/Data/hf50_h143_cl2withEnc_c1_143.train', 'rb'), encoding='bytes')
valid_sl = pickle.load(open('/data/projects/py_ehr_2/Data/hf50_h143_cl2withEnc_c1_143.valid', 'rb'), encoding='bytes')
test_sl = pickle.load(open('/data/projects/py_ehr_2/Data/hf50_h143_cl2withEnc_c1_143.test', 'rb'), encoding='bytes')

# Readmission no encounter input_size= 8690
in_size= 8690
train_sl = pickle.load(open('/data/projects/Readm/pdata/readm_diag_comb_h143.train', 'rb'), encoding='bytes')
valid_sl = pickle.load(open('/data/projects/Readm/pdata/readm_diag_comb_h143.valid', 'rb'), encoding='bytes')
test_sl =  pickle.load(open('/data/projects/Readm/pdata/readm_diag_comb_h143.test', 'rb'), encoding='bytes')

'''
# Readmission with encounter input_size= 12433
in_size= 12433
train_sl = pickle.load(open('/data/projects/Readm/pdata/readm_diag_enc_comb_h143.train', 'rb'), encoding='bytes')
valid_sl = pickle.load(open('/data/projects/Readm/pdata/readm_diag_enc_comb_h143.valid', 'rb'), encoding='bytes')
test_sl =  pickle.load(open('/data/projects/Readm/pdata/readm_diag_enc_comb_h143.test', 'rb'), encoding='bytes')


In [4]:
class EHR_RNN(nn.Module):
    def __init__(self, input_size,embed_dim, hidden_size, n_layers=1,dropout_r=0.1,cell_type='LSTM',bi=False , preTrainEmb=''):
        super(EHR_RNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embed_dim = embed_dim
        self.dropout_r = dropout_r
        self.cell_type = cell_type
        self.preTrainEmb=preTrainEmb
        if bi: self.bi=2 
        else: self.bi=1
              
        if len(self.preTrainEmb)>0:
            emb_t= torch.FloatTensor(np.asmatrix(self.preTrainEmb))
            self.embed= nn.Embedding.from_pretrained(emb_t)#,freeze=False)  
        else:
            self.embed= nn.Embedding(input_size, self.embed_dim,padding_idx=0)
        
        if self.cell_type == "GRU":
            cell = nn.GRU
        elif self.cell_type == "RNN":
            cell = nn.RNN
        elif self.cell_type == "LSTM":
            cell = nn.LSTM
        else:
            raise NotImplementedError
      
        self.rnn_c = cell(self.embed_dim, hidden_size,num_layers=n_layers, dropout= dropout_r , bidirectional=bi , batch_first=True )
        self.out = nn.Linear(self.hidden_size*self.bi,1)
        self.sigmoid = nn.Sigmoid()

        
    def EmbedPatient_MB(self, input): # x is a ehr_seq_tensor
        
        mb=[]
        lbt=[]
        seq_l=[]
        self.bsize=len(input)
        lp= len(max(input, key=lambda xmb: len(xmb[1]))[1])
        #print (lp)
        llv=0
        for x in input:
            lv= len(max(x[1], key=lambda xmb: len(xmb)))
            if llv< lv:
                llv=lv
        #print (llv)
        for pt in input:
            label, ehr_seq_l = pt
            lpx=len(ehr_seq_l)
            seq_l.append(lpx)
            label_tensor = Variable(torch.FloatTensor([[float(label)]]))
            if use_cuda:
                label_tensor = label_tensor.cuda()
            lbt.append(label_tensor)
            ml=(len(max(ehr_seq_l, key=len)))
            ehr_seq_tl=[]
            for ehr_seq in ehr_seq_l: 
                pd=(0, llv-len(ehr_seq))
                result = F.pad(torch.from_numpy(np.asarray(ehr_seq,dtype=int)).type(torch.cuda.LongTensor),pd,"constant", 0)
                if use_cuda:
                    result.cuda()
                ehr_seq_tl.append(result)
            ehr_seq_t= Variable(torch.stack(ehr_seq_tl,0))     
            lpp= lp-lpx
            zp= nn.ZeroPad2d((0,0,0,lpp))
            ehr_seq_t= zp(ehr_seq_t)
            mb.append(ehr_seq_t)
                
        mb_t= Variable(torch.stack(mb,0)) 
        if use_cuda:
            mb_t.cuda()
        embedded = self.embed(mb_t)
        embedded = torch.sum(embedded, dim=2) 
        lbt_t= Variable(torch.stack(lbt,0)) 
        return embedded, lbt_t,seq_l
    
    def init_hidden(self):
        
        h_0 = Variable(torch.rand(self.n_layers*self.bi,self.bsize, self.hidden_size))

        if use_cuda:
            h_0 =h_0.cuda()
        if self.cell_type == "LSTM":
            result = (h_0,h_0)
        else: 
            result = h_0
            
        return result
    
    def forward(self, input):
        
        x_in , lt ,x_lens = self.EmbedPatient_MB(input)
        x_inp = nn.utils.rnn.pack_padded_sequence(x_in,x_lens,batch_first=True)
        h_0 = self.init_hidden()
        output, hidden = self.rnn_c(x_inp,h_0) 
        if self.cell_type == "LSTM":
            hidden=hidden[0]
        if self.bi==2:
            output = self.sigmoid(self.out(torch.cat((hidden[-2],hidden[-1]),1)))
        else:
            output = self.sigmoid(self.out(hidden[-1]))
        return output.squeeze(), lt.squeeze()

In [5]:
def train (tmodel,mini_batch, criterion, optimizer):  
    
    tmodel.train()
    tmodel.zero_grad()
    output , label_tensor = tmodel(mini_batch)
    loss = criterion(output, label_tensor)
    loss.backward()
    optimizer.step()
   
    return output, loss.item()

In [6]:
# training all samples in random order
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [7]:
def run_model_train(tmodel,dataset,batch_size,learning_rate = 0.01, l2=1e-04,epsl=1e-06 ):
    
    #optimizer = optim.SGD(tmodel.parameters(), lr=learning_rate)#, weight_decay=l2)
    #optimizer = optim.Adadelta(tmodel.parameters(), lr=learning_rate, weight_decay=l2)
    #optimizer = optim.ASGD(tmodel.parameters(), lr=learning_rate, weight_decay=l2 )
    #optimizer = optim.SparseAdam (tmodel.parameters(),lr=learning_rate) #'''lr=learning_rate,''' 
    optimizer = optim.Adagrad (tmodel.parameters(),lr=learning_rate, weight_decay=l2) #'''lr=learning_rate,''' 
    #optimizer = optim.Adamax(tmodel.parameters(), lr=learning_rate, weight_decay=l2 ,eps=epsl)
    #optimizer = optim.Adamax(filter(lambda p: p.requires_grad, tmodel.parameters()), lr=learning_rate, weight_decay=l2 ,eps=epsl) ### Beta defaults (0.9, 0.999)
    #optimizer = optim.RMSprop (tmodel.parameters(),lr=learning_rate, weight_decay=l2 ,eps=epsl)
    #optimizer = optim.Adam(tmodel.parameters(), lr=learning_rate, weight_decay=learning_rate)
    dataset.sort(key=lambda pt:len(pt[1]),reverse=True) 
    # Keep track of losses for plotting
    current_loss = 0
    all_losses = []
    print_every = 10#int(batch_size/2)
    plot_every = 5
    iter=0
    n_batches = int(np.ceil(int(len(dataset)) / int(batch_size)))
    start = time.time()

    for index in random.sample(range(n_batches), n_batches):
            batch = dataset[index*batch_size:(index+1)*batch_size]
            output, loss = train(tmodel,batch, criterion = nn.BCELoss(), optimizer = optimizer)
            current_loss += loss
            iter +=1
            # Add current loss avg to list of losses
            if iter % plot_every == 0:
                all_losses.append(current_loss / plot_every)
                current_loss = 0
                
    return current_loss,all_losses


In [8]:
def calculate_auc(test_model, dataset, batch_size=200):
    test_model.eval()
    dataset.sort(key=lambda pt:len(pt[1]),reverse=True) 
    n_batches = int(np.ceil(int(len(dataset)) / int(batch_size)))
    labelVec =[]
    y_hat= []
    
    for index in range(n_batches):
            batch = dataset[index*batch_size:(index+1)*batch_size]
            output, label_t = test_model(batch)
            y_hat.extend(output.cpu().data.view(-1).numpy())
            labelVec.extend(label_t.cpu().data.view(-1).numpy())
    auc = roc_auc_score(labelVec, y_hat)
    
    return auc

In [9]:
epochs=100
batch_size=128
current_loss_l=[]
all_losses_l=[]
train_auc_allep =[]
valid_auc_allep =[]
test_auc_allep=[]
bestValidAuc = 0.0
bestTestAuc = 0.0
bestValidEpoch = 0

### define the new model

emb_pretrain=''                       
###HF no encounter 15816, with encounter 18250 , Readm without enc 8690 , Readm with enc 12433 (in_size)
model = EHR_RNN(input_size=in_size, hidden_size=128 ,embed_dim=256, dropout_r=0, cell_type='GRU',bi=False , n_layers=1, preTrainEmb=emb_pretrain)
bmodel_pth = './best_model_BiLSTM.pth'
bmodel_st = './best_model_BiLSTM.st'
if use_cuda:
    model = model.cuda()

### Run Epochs    
for ep in range(epochs):
    
    #print (model.embed.weight.data[135] )
    start = time.time()
    current_loss_la,all_losses_la = run_model_train(model,train_sl,batch_size)
    train_time = timeSince(start)
    eval_start = time.time()
    train_auc = calculate_auc(model,train_sl,batch_size)
    test_auc = calculate_auc(model,test_sl,batch_size)
    valid_auc = calculate_auc(model,valid_sl,batch_size)
    eval_time = timeSince(eval_start)
    all_losses_l.append (all_losses_la)
    avg_loss = np.mean(all_losses_la)
    train_auc_allep.append(train_auc)
    valid_auc_allep.append(valid_auc)
    test_auc_allep.append(test_auc)
    current_loss_l.append(current_loss_la)
    print ("Epoch ", ep," Train_auc :", train_auc, " , Valid_auc : ", valid_auc, " ,& Test_auc : " , test_auc," Avg Loss: ", avg_loss, 'Train Time (%s) Eval Time (%s)'%(train_time,eval_time) )
     
    if valid_auc > bestValidAuc: 
        bestValidAuc = valid_auc
        bestValidEpoch = ep
        bestTestAuc = test_auc
        best_model = model
        torch.save(best_model, bmodel_pth)
        torch.save(best_model.state_dict(), bmodel_st)
    if ep - bestValidEpoch >12: break
            
print ('bestValidAuc %f has a TestAuc of %f at epoch %d ' % (bestValidAuc, bestTestAuc, bestValidEpoch))



Epoch  0  Train_auc : 0.7115027026599061  , Valid_auc :  0.6450256651870956  ,& Test_auc :  0.6671947338950875  Avg Loss:  0.6798904563251295 Train Time (0m 38s) Eval Time (0m 53s)


/home/lgindybekhet/.local/lib/python3.5/site-packages/torch/serialization.py:193: UserWarning:

Couldn't retrieve source code for container of type EHR_RNN. It won't be checked for correctness upon loading.



Epoch  1  Train_auc : 0.7402763978799356  , Valid_auc :  0.6648200957175093  ,& Test_auc :  0.6896989954721131  Avg Loss:  0.6466100517072175 Train Time (0m 46s) Eval Time (1m 4s)
Epoch  2  Train_auc : 0.7659130698870945  , Valid_auc :  0.6749225084930692  ,& Test_auc :  0.6952243023491127  Avg Loss:  0.62762205726222 Train Time (1m 18s) Eval Time (1m 20s)
Epoch  3  Train_auc : 0.7909376314483805  , Valid_auc :  0.6748121605871996  ,& Test_auc :  0.7005390134495744  Avg Loss:  0.6154361982094613 Train Time (0m 37s) Eval Time (0m 52s)
Epoch  4  Train_auc : 0.8156408676973573  , Valid_auc :  0.686641704069234  ,& Test_auc :  0.7043576720181066  Avg Loss:  0.5958315849304199 Train Time (0m 38s) Eval Time (0m 50s)
Epoch  5  Train_auc : 0.8408102001101121  , Valid_auc :  0.6922917648225755  ,& Test_auc :  0.7049672668672163  Avg Loss:  0.5836552949328171 Train Time (0m 45s) Eval Time (1m 3s)
Epoch  6  Train_auc : 0.8645343616217016  , Valid_auc :  0.6931770277977534  ,& Test_auc :  0.705892

In [10]:
import plotly.plotly as py 
import plotly.graph_objs as go
py.sign_in('LailaRasmy','mzNHzVvwYjcZwBDZx3B7')

train_auc_fg= go.Scatter(x= np.arange(epochs), y=train_auc_allep, name='train')
test_auc_fg= go.Scatter(x= np.arange(epochs), y=test_auc_allep, name='test')
valid_auc_fg= go.Scatter(x= np.arange(epochs), y=valid_auc_allep, name='valid')
valid_max = max(valid_auc_allep)
test_max = max(test_auc_allep)
data = [train_auc_fg,test_auc_fg,valid_auc_fg]
layout = go.Layout(xaxis=dict(dtick=1))
layout.update(dict(annotations=[go.Annotation(text="Max Valid", x=valid_auc_allep.index(valid_max), y=valid_max)]))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='BiRNN_Auc')